In [16]:
from math import *
import random
import abc

In [17]:
class ActivationF(abc.ABC):

    @abc.abstractmethod
    def calc(self, x):
        pass

    @abc.abstractmethod
    def derive(self, x):
        pass

    @staticmethod
    def from_name(name="tanh"):
        if name == "tanh":
            return TanhF()

In [18]:
class TanhF(ActivationF):

  def calc(self, x):
        act_1 = (exp(x) - exp(-x))/(exp(x) + exp(-x))
        return act_1
  
  def derive(self, x):
        act_2 = 1-calc(x)**2
        return act_2

In [20]:
TanhF().calc(2)

0.964027580075817

In [19]:
# def matmult(a,b):
#     zip_b = zip(*b)
#     # uncomment next line if python 3 : 
#     zip_b = list(zip_b)
#     return [[sum(ele_a*ele_b for ele_a, ele_b in zip(row_a, col_b)) 
#              for col_b in zip_b] for row_a in a]

In [45]:
class Unit:
    def __init__(self, input_size, prev_layer: "Layer" = None, activation="tanh"):
        self.activation = ActivationF.from_name(activation)
        self.weights = [random.uniform(-1, 1) for weight_n in range(input_size)]
        self.prev_layer = prev_layer
        #self.output_size = output_size
        self.input_size = input_size
        self.w_mid = sum([(x+1)*int(self.weights[x])for x in range(len(self.weights))])
      
    def forward(self):
        result = 0
        if self.prev_layer is not None:
          m_l = self.prev_layer.forward()
        else:
          m_l = self.features
          
        for w, prev in zip(self.weights, m_l):
          result += w * prev

        return self.activation.calc(result)

    def backward(self, losses, learning_rate=0.01):
        
        #losses = 

        if self.prev_layer is not None:
          m_l = self.prev_layer.backward()
        else:
          m_l = self.features
        
        error = losses * self.activation.derive(w_mid)
        print(m_l)
        self.n_weights = []
        for wei, feat in zip(self.weights, m_l):
          n_w = wei * error + feat
          self.n_weights.append(n_w)

In [46]:
class Layer:
    def __init__(self, input_size, size, prev_layer = None, activation="tanh"):
        self.size = size
        self.input_size = input_size
        self.activation = activation
        self.units = [Unit(self.input_size, 
                           prev_layer=prev_layer, 
                           #output_size=self.output_size, 
                           activation=self.activation)
                      for unit_n in range(self.size)]
    
    def forward(self):
        return [unit.forward() for unit in self.units]

    def backward(self, losses):
        loss_units = []
        for u, l in zip(self.units, losses):
          loss_units.append(u.backward(l))
        return loss_units
        #return [unit.backward() for unit in self.units]

In [101]:
class MLP:
    def __init__(self, input_size, output_size, sizes, activation="tanh"):
        self.activation = activation
        self.losses = []
        layers_sizes = [input_size] + sizes + [output_size]
        
        self.layers = []
        prev_added_layer = None
        for layer_in_size, layer_out_size in zip(layers_sizes[:-1], layers_sizes[1:]):
            self.layers.append(Layer(layer_in_size, layer_out_size, prev_layer=prev_added_layer))
            prev_added_layer = self.layers[-1]
    
    def train_single_entry(self, features, target_mhe):
        # region forward pass
        self.features = features
        for unit in self.layers[0].units:
            unit.features = self.features

        for layer in self.layers:
            curr_layer_output = layer.forward()
        last_layer_output = curr_layer_output
        # endregion forward pass
        # лосс с софтмаксом удобно считать вместе -- 
        # уж больно хорошая получается производная ошибки
        # по выходу с последнего лося
        # но адекватнее было бы наверное реализовать его тоже как наследника ActivationF
        
        def softmax(some_data):
            es_x = [e ** x for x in some_data]
            return [e_x / sum(es_x) for e_x in es_x]

        pred = softmax(last_layer_output)
        loss = - sum(class_target * log(class_pred)
                     for class_target, class_pred in zip(target_mhe, pred))
        #self.losses.append(loss)
        # region backward pass
        dLoss_dLastLayerOutput = [class_pred - class_target
                                  for class_target, class_pred in zip(target_mhe, pred)]
        
        # your code here
        # endregion backward pass
        #self.losses.append(loss)
        #print(dLoss_dLastLayerOutput)
        return loss
        
    # your code here

In [102]:
mlp = MLP(6,5,[8, 7, 15, 137])

In [103]:
mlp.train_single_entry([11,14, 45, 15],[17,18,16])

-16.801802608746513